In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

In [5]:
df = pd.read_csv(r"C:\Users\HP\Downloads\Medicinal Plant identification\Medicinal Plant identification\medicinal_plants_single_disease.csv")




In [6]:
# Drop missing values
df = df.dropna(subset=["Disease Cured", "Plant Name"])


In [7]:

# Text and labels
texts = df["Disease Cured"].astype(str).tolist()
labels = df["Plant Name"].astype(str).tolist()

In [8]:
# Label encoding
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

In [9]:

# Save label list to use in Android
label_list = label_encoder.classes_.tolist()
with open("label_list.json", "w") as f:
    json.dump(label_list, f)


In [10]:
# Tokenization
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, padding="post")


In [11]:

# Save tokenizer to JSON
token_json = tokenizer.to_json()
with open("tokenizer.json", "w") as f:
    f.write(token_json)


In [12]:

# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16, input_length=padded_sequences.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [13]:

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 16)             80000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 32)                544       
                                                                 
 dense_1 (Dense)             (None, 33)                1089      
                                                                 
Total params: 81633 (318.88 KB)
Trainable params: 81633 (318.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:


# Train the model
model.fit(padded_sequences, labels_encoded, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10


2/2 [==============================] - 1s 295ms/step - loss: 3.4959 - accuracy: 0.0769 - val_loss: 3.4942 - val_accuracy: 0.1429
Epoch 2/10
2/2 [==============================] - 0s 144ms/step - loss: 3.4934 - accuracy: 0.0385 - val_loss: 3.4962 - val_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 0s 104ms/step - loss: 3.4918 - accuracy: 0.1154 - val_loss: 3.4981 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 0s 86ms/step - loss: 3.4902 - accuracy: 0.1154 - val_loss: 3.5000 - val_accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 0s 71ms/step - loss: 3.4887 - accuracy: 0.1154 - val_loss: 3.5020 - val_accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 0s 62ms/step - loss: 3.4871 - accuracy: 0.1154 - val_loss: 3.5039 - val_accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 0s 61ms/step - loss: 3.4854 - accuracy: 0.1154 - val_loss: 3.5059 - val_accuracy: 0.0000e+0

In [15]:
# Save the model
model.save("plant_recommendation_model.h5")


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
 model = tf.keras.models.load_model("plant_recommendation_model.h5")

    
#Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open("plant_recommendation_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpimvpct4h\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpimvpct4h\assets


In [ ]:




# Load CSV dataset
file_path = "medicinal_plants_single_disease.csv"
df = pd.read_csv(file_path)











In [17]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

In [18]:
df = pd.read_csv(r"C:\Users\HP\Downloads\Medicinal Plant identification\Medicinal Plant identification\medicinal_plants_single_disease.csv")




In [19]:
# Drop missing values
df = df.dropna(subset=["Disease Cured", "Plant Name"])

In [20]:
# Texts and labels
texts = df["Disease Cured"].astype(str).tolist()
labels = df["Plant Name"].astype(str).tolist()

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)


In [21]:
# Save label list for Android
with open("label_list.json", "w") as f:
    json.dump(label_encoder.classes_.tolist(), f)


In [22]:
# Tokenize the text
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, padding="post")

In [23]:
# Save tokenizer to JSON
with open("tokenizer.json", "w") as f:
    f.write(tokenizer.to_json())


In [24]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels_encoded, test_size=0.2, random_state=42)


In [25]:
# Build improved model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=1000, output_dim=32, input_length=padded_sequences.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [26]:


model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 32)             32000     
                                                                 
 global_average_pooling1d (  (None, 32)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 batch_normalization (Batch  (None, 32)                128       
 Normalization)                                                  
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 33)               

In [27]:




# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/25
7/7 [==============================] - 3s 176ms/step - loss: 3.4617 - accuracy: 0.1154 - val_loss: 3.5085 - val_accuracy: 0.0000e+00
Epoch 2/25
7/7 [==============================] - 0s 17ms/step - loss: 3.3858 - accuracy: 0.1923 - val_loss: 3.5162 - val_accuracy: 0.0000e+00
Epoch 3/25
7/7 [==============================] - 0s 16ms/step - loss: 3.3789 - accuracy: 0.0385 - val_loss: 3.5241 - val_accuracy: 0.0000e+00
Epoch 4/25
7/7 [==============================] - 0s 9ms/step - loss: 3.2036 - accuracy: 0.2308 - val_loss: 3.5335 - val_accuracy: 0.0000e+00
Epoch 5/25
7/7 [==============================] - 0s 10ms/step - loss: 3.1874 - accuracy: 0.1923 - val_loss: 3.5453 - val_accuracy: 0.0000e+00
Epoch 6/25
7/7 [==============================] - 0s 10ms/step - loss: 3.0052 - accuracy: 0.4615 - val_loss: 3.5589 - val_accuracy: 0.0000e+00
Epoch 7/25
7/7 [==============================] - 0s 11ms/step - loss: 2.9654 - accuracy: 0.4615 - val_loss: 3.5746 - val_accuracy: 0.0000e+00

In [28]:

# Save the model
model.save("plant_recommendation_model.h5")


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("plant_recommendation_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmp5j9po1z8\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmp5j9po1z8\assets


In [31]:
# Predict function (for testing in Python)
def predict_plant(disease):
    seq = tokenizer.texts_to_sequences([disease])
    pad_seq = pad_sequences(seq, maxlen=padded_sequences.shape[1], padding="post")
    pred = model.predict(pad_seq)
    plant = label_encoder.inverse_transform([pred.argmax()])[0]
    return plant

In [ ]:

# Example use
while True:
    query = input("Enter a disease (or type 'exit'): ").strip()
    if query.lower() == 'exit':
        break
    result = predict_plant(query)
    print(f"Recommended Plant: {result}\n")



Enter a disease (or type 'exit'): cough
1/1 [==============================] - 0s 125ms/step
Recommended Plant: Fenugreek

Enter a disease (or type 'exit'): tulsi
1/1 [==============================] - 0s 22ms/step
Recommended Plant: Fenugreek



In [ ]:








# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("plant_recommendation_model.tflite", "wb") as f:
    f.write(tflite_model)


In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import class_weight
import numpy as np
import json

In [2]:


# Load the dataset
file_path = "medicinal_plants_single_disease.csv"
df = pd.read_csv(file_path)

In [3]:

# Drop missing values
df = df.dropna(subset=["Disease Cured", "Plant Name", "Medicinal Benefits"])

In [4]:

# Combine Disease Cured and Medicinal Benefits for better context
df['Combined'] = df['Disease Cured'] + ' ' + df['Medicinal Benefits']
texts = df['Combined'].astype(str).tolist()
labels = df['Plant Name'].astype(str).tolist()


In [6]:

# Label encoding
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

In [7]:


# Save label list to use in Android
label_list = label_encoder.classes_.tolist()
with open("label_list.json", "w") as f:
    json.dump(label_list, f)


In [8]:


# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, padding="post")


In [9]:


# Save tokenizer
token_json = tokenizer.to_json()
with open("tokenizer.json", "w") as f:
    f.write(token_json)

In [10]:

# Calculate class weights to reduce bias
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(labels_encoded), y=labels_encoded)
class_weights_dict = dict(enumerate(class_weights))


In [11]:
# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=32, input_length=padded_sequences.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [12]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 14, 32)            320000    
                                                                 
 bidirectional (Bidirection  (None, 14, 128)           49664     
 al)                                                             
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 33)                

In [13]:
# Train the model
model.fit(padded_sequences, labels_encoded, epochs=15, batch_size=32, validation_split=0.2, class_weight=class_weights_dict)

Epoch 1/15


1/1 [==============================] - 15s 15s/step - loss: 3.4973 - accuracy: 0.0000e+00 - val_loss: 3.4982 - val_accuracy: 0.0000e+00
Epoch 2/15
1/1 [==============================] - 0s 120ms/step - loss: 3.4948 - accuracy: 0.0385 - val_loss: 3.5027 - val_accuracy: 0.0000e+00
Epoch 3/15
1/1 [==============================] - 0s 136ms/step - loss: 3.4918 - accuracy: 0.0769 - val_loss: 3.5072 - val_accuracy: 0.0000e+00
Epoch 4/15
1/1 [==============================] - 0s 127ms/step - loss: 3.4882 - accuracy: 0.1154 - val_loss: 3.5117 - val_accuracy: 0.0000e+00
Epoch 5/15
1/1 [==============================] - 0s 109ms/step - loss: 3.4876 - accuracy: 0.0385 - val_loss: 3.5159 - val_accuracy: 0.0000e+00
Epoch 6/15
1/1 [==============================] - 0s 164ms/step - loss: 3.4838 - accuracy: 0.1923 - val_loss: 3.5203 - val_accuracy: 0.0000e+00
Epoch 7/15
1/1 [==============================] - 0s 136ms/step - loss: 3.4860 - accuracy: 0.0769 - val_loss: 3.5245 - val_accuracy

In [14]:

# Save the model
model.save("plant_recommendation_model.h5")


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:

# --- Prediction Function (Desktop Test) ---
print("\nModel ready! Test below.\n")

while True:
    query = input("Enter a disease (or type 'exit'): ").strip().lower()
    if query == 'exit':
        break

    # Preprocess and predict
    sequence = tokenizer.texts_to_sequences([query])
    padded = pad_sequences(sequence, maxlen=padded_sequences.shape[1], padding='post')
    prediction = model.predict(padded)
    predicted_index = np.argmax(prediction)
    plant_name = label_encoder.inverse_transform([predicted_index])[0]
    print(f"Recommended Plant: {plant_name}\n")



Model ready! Test below.

Enter a disease (or type 'exit'): cough
1/1 [==============================] - 3s 3s/step
Recommended Plant: Betel

Enter a disease (or type 'exit'): cold
1/1 [==============================] - 0s 85ms/step
Recommended Plant: Betel

Enter a disease (or type 'exit'): fever
1/1 [==============================] - 0s 66ms/step
Recommended Plant: Betel



In [ ]:






# Preparing data for model training (Supervised learning)
df['Label'] = df['Plant Name'].astype('category').cat.codes
X_train, X_test, y_train, y_test = train_test_split(df['Content'], df['Label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()

# Build a neural network model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(df['Label'].unique()), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_tfidf, y_train, epochs=10, validation_data=(X_test_tfidf, y_test))

# Save the trained model to .h5 format
model.save("plant_recommendation_model.h5")

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open("plant_recommendation_model.tflite", "wb") as f:
    f.write(tflite_model)

# Save the vocab and labels as JSON (for inference in Android)
import json

# Save vocab
with open("vocab.json", "w") as vocab_file:
    json.dump(vectorizer.get_feature_names_out().tolist(), vocab_file)

# Save labels (mapping plant names to labels)
labels_dict = dict(zip(df['Label'].unique(), df['Plant Name'].unique()))
with open("labels.json", "w") as labels_file:
    json.dump(labels_dict, labels_file)

# Prediction function using the TFLite model
def predict_plant_tflite(disease_query, model_file="plant_recommendation_model.tflite", vocab_file="vocab.json", labels_file="labels.json"):
    # Load vocab and labels
    with open(vocab_file, "r") as vocab_file:
        vocab = json.load(vocab_file)
    with open(labels_file, "r") as labels_file:
        labels = json.load(labels_file)

    # Convert disease query to TF-IDF vector using vocab
    query_vector = [0] * len(vocab)
    for word in disease_query.split():
        if word in vocab:
            query_vector[vocab.index(word)] = 1  # Simulate TF-IDF vector (simplified)

    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=model_file)
    interpreter.allocate_tensors()

    # Get input tensor details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Set input tensor
    input_index = input_details[0]['index']
    interpreter.set_tensor(input_index, [query_vector])

    # Run inference
    interpreter.invoke()

    # Get prediction result
    output_index = output_details[0]['index']
    output = interpreter.get_tensor(output_index)

    # Get the predicted plant name
    predicted_label = output.argmax(axis=1)[0]  # Get the label with highest probability
    predicted_plant = labels.get(str(predicted_label), "Unknown Plant")

    return predicted_plant

# Example: Test the prediction
disease_query = input("Enter disease for prediction: ")
recommended_plant = predict_plant_tflite(disease_query)
print(f"Recommended Plant: {recommended_plant}")


In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split


In [4]:

# Load dataset
df = pd.read_csv(r"C:\Users\HP\Downloads\Medicinal Plant identification\Medicinal Plant identification\medicinal_plants_single_disease.csv")

In [5]:


# Combine 'Disease Cured' and 'Medicinal Benefits' to form the content for vectorization
df['Content'] = df['Disease Cured'] + " " + df['Medicinal Benefits']
df = df.dropna(subset=['Content'])
df['Content'] = df['Content'].fillna('')

In [6]:


# TF-IDF Vectorizer to transform content into numerical representation
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Content'])

In [7]:
# Function to load data (optional as you already load it in the beginning)
def load_data():
    file_path = r"C:\Users\HP\Downloads\Medicinal Plant identification\Medicinal Plant identification\medicinal_plants_single_disease.csv"
    df = pd.read_csv(file_path)
    return df


In [8]:
# Function to train the model
def train_model(df):
    vectorizer = TfidfVectorizer(max_features=5000)
    tfidf_matrix = vectorizer.fit_transform(df['Content'])
    return vectorizer, tfidf_matrix


In [10]:
# Function for plant recommendation based on disease input
def recommend_plant(disease_query, df, vectorizer, tfidf_matrix):
    query_tfidf = vectorizer.transform([disease_query])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    
    best_match_idx = cosine_similarities.argmax()  # Get the best match
    best_match = df.iloc[best_match_idx]
    
    print(f"Best recommendation for '{disease_query}':\n")
    print(f"Plant Name: {best_match['Plant Name']}")
    print(f"Disease Cured: {best_match['Disease Cured']}")
    print(f"Benefits: {best_match['Medicinal Benefits']}")
    print(f"Similarity Score: {cosine_similarities[best_match_idx]:.2f}")
    print("-" * 40)

In [12]:

# Prepare labels for supervised training
df['Label'] = df['Plant Name'].astype('category').cat.codes
X_train, X_test, y_train, y_test = train_test_split(df['Disease Cured'], df['Label'], test_size=0.2, random_state=42)

In [13]:
# TF-IDF vectorizer for model input
# ===================== TF Model Training =====================

model_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = model_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = model_vectorizer.transform(X_test).toarray()

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(df['Label'].unique()), activation='softmax')
])


In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [15]:

# Train the model
model.fit(X_train_tfidf, y_train, epochs=10, validation_data=(X_test_tfidf, y_test))


Epoch 1/10


1/1 [==============================] - 3s 3s/step - loss: 3.4829 - accuracy: 0.0769 - val_loss: 3.5212 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 107ms/step - loss: 3.4678 - accuracy: 0.0769 - val_loss: 3.5274 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 106ms/step - loss: 3.4526 - accuracy: 0.0769 - val_loss: 3.5336 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 141ms/step - loss: 3.4375 - accuracy: 0.0769 - val_loss: 3.5399 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 109ms/step - loss: 3.4223 - accuracy: 0.1154 - val_loss: 3.5463 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 100ms/step - loss: 3.4072 - accuracy: 0.1538 - val_loss: 3.5528 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 104ms/step - loss: 3.3921 - accuracy: 0.1923 - val_loss: 3.5594 - val_accuracy: 0.00

In [16]:
# Save the model to H5
model.save("plant_recommendation_model.h5")


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
# Convert to TFLite
model = tf.keras.models.load_model("plant_recommendation_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save TFLite model
with open("plant_recommendation_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmp7baj63q9\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmp7baj63q9\assets


In [18]:
# ===================== Main Code =====================

if __name__ == "__main__":
    disease_query = input("Enter the disease or symptom for recommendation: ")
    cosine_vectorizer, cosine_matrix = train_cosine_model(df)
    recommend_plant_cosine(disease_query, df, cosine_vectorizer, cosine_matrix)

Enter the disease or symptom for recommendation: cough


NameError: name 'train_cosine_model' is not defined

In [ ]:

#

# ===================== Cosine Similarity Recommendation =====================




# ===================== TF Model Training =====================









In [19]:


import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from sklearn.model_selection import train_test_split







In [20]:
#Load dataset
df = pd.read_csv(r"C:\Users\HP\Downloads\Medicinal Plant identification\Medicinal Plant identification\medicinal_plants_single_disease.csv")


In [21]:
# Combine 'Disease Cured' and 'Medicinal Benefits' for cosine similarity
df['Content'] = df['Disease Cured'].fillna('') + " " + df['Medicinal Benefits'].fillna('')
df = df.dropna(subset=['Content'])


In [22]:
def train_cosine_model(df):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df['Content'])
    return vectorizer, tfidf_matrix

In [23]:
def recommend_plant_cosine(disease_query, df, vectorizer, tfidf_matrix):
    query_tfidf = vectorizer.transform([disease_query])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    best_match_idx = cosine_similarities.argmax()
    best_match = df.iloc[best_match_idx]

    print("\n🔍 Cosine Similarity Recommendation:")
    print(f"Best match for '{disease_query}':")
    print(f"🌿 Plant Name: {best_match['Plant Name']}")
    print(f"💊 Disease Cured: {best_match['Disease Cured']}")
    print(f"📋 Benefits: {best_match['Medicinal Benefits']}")
    print(f"📈 Similarity Score: {cosine_similarities[best_match_idx]:.2f}")
    print("-" * 40)


In [24]:
# Prepare labels for supervised training
df['Label'] = df['Plant Name'].astype('category').cat.codes
X_train, X_test, y_train, y_test = train_test_split(df['Disease Cured'], df['Label'], test_size=0.2, random_state=42)

In [25]:
# TF-IDF vectorizer for model input
model_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = model_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = model_vectorizer.transform(X_test).toarray()


In [26]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(df['Label'].unique()), activation='softmax')
])


In [27]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [28]:
# Train the model
model.fit(X_train_tfidf, y_train, epochs=10, validation_data=(X_test_tfidf, y_test))

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 3.4973 - accuracy: 0.0769 - val_loss: 3.4913 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 118ms/step - loss: 3.4818 - accuracy: 0.0769 - val_loss: 3.4968 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 114ms/step - loss: 3.4663 - accuracy: 0.0769 - val_loss: 3.5022 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 119ms/step - loss: 3.4508 - accuracy: 0.1154 - val_loss: 3.5075 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 116ms/step - loss: 3.4354 - accuracy: 0.1154 - val_loss: 3.5129 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 174ms/step - loss: 3.4201 - accuracy: 0.1538 - val_loss: 3.5184 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 201ms/step - loss: 3.4048 - accuracy: 0.1923 - val_loss: 3.5239 - val_accuracy: 0.0000

In [29]:

# Save the model to H5
model.save("plant_recommendation_model.h5")


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:


# Convert to TFLite
model = tf.keras.models.load_model("plant_recommendation_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmptr3tncae\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmptr3tncae\assets


In [31]:

# Save TFLite model
with open("plant_recommendation_model.tflite", "wb") as f:
    f.write(tflite_model)

In [34]:



# ===================== Main Code =====================

if __name__ == "__main__":
    disease_query = input("Enter the disease or symptom for recommendation: ")
    cosine_vectorizer, cosine_matrix = train_cosine_model(df)
    recommend_plant_cosine(disease_query, df, cosine_vectorizer, cosine_matrix)

Enter the disease or symptom for recommendation: skin infection

🔍 Cosine Similarity Recommendation:
Best match for 'skin infection':
🌿 Plant Name: Sandalwood
💊 Disease Cured: Skin disorders
📋 Benefits: Soothes skin irritation, promotes relaxation, antimicrobial properties.
📈 Similarity Score: 0.56
----------------------------------------


In [35]:
# At the end of your script, export the data
recommendation_data = df[['Plant Name', 'Disease Cured', 'Medicinal Benefits', 'Content']]
recommendation_data.to_json("recommendation_data.json", orient="records", lines=False)
